In [89]:
import pandas as pd
import random
import time
import re
import warnings
warnings.filterwarnings('ignore')
from selenium import webdriver  # 동적크롤링
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

from urllib.parse import quote

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [90]:
df = pd.read_csv("Data/제주도식당_전처리.csv", index_col=False)

In [91]:
df.head()

,사업장명,업종구분대분류,업종구분소분류,소재지전체주소,도로명전체주소,lat,lng
0,탑동에이치지,일반음식점,기타,제주특별자치도 제주시 삼도이동 1120-2,"제주특별자치도 제주시 무근성안길 16, 1층 (삼도이동)",33.5151128,126.5202001
1,에릭스에스프레소,일반음식점,기타,제주특별자치도 제주시 구좌읍 세화리 1397-6번지,"제주특별자치도 제주시 구좌읍 구좌로 77, 1층",33.5214225,126.8588743
2,일품순두부한림점,일반음식점,한식,제주특별자치도 제주시 한림읍 대림리 1845-2번지,"제주특별자치도 제주시 한림읍 한림상로 237, 1층",33.4192601,126.2674304
3,김복남맥주제주아라점,일반음식점,식육(숯불구이),제주특별자치도 제주시 아라일동 6139,"제주특별자치도 제주시 중앙로 581, 에이동 1층 (아라일동)",33.4714278,126.5457848
4,봉플라봉뱅,일반음식점,기타,제주특별자치도 제주시 연동 312-57번지 정인하우스,"제주특별자치도 제주시 문송1길 6-1, 정인하우스 1층 (연동)",33.4879976,126.4978933


# 현재 없는데이터는 17001~19000까지의 데이터.

In [92]:
df.count()

사업장명       20656
업종구분대분류    20656
업종구분소분류    20653
소재지전체주소    20580
도로명전체주소    20656
lat        20656
lng        20656
dtype: int64

In [93]:
df['사업장명']
scrapy_df2 = "제주 " + df['사업장명']
scrapy_df2

0            제주 탑동에이치지
1          제주 에릭스에스프레소
2          제주 일품순두부한림점
3        제주 김복남맥주제주아라점
4             제주 봉플라봉뱅
             ...      
20651       제주 못(MOAT)
20652      제주 제주하이루탕후루
20653            제주 연돈
20654        제주 가시림 카페
20655          제주 올드타운
Name: 사업장명, Length: 20656, dtype: object

### 크롤링에 필요한 컬럼만 뽑아서 저장하기

In [94]:
scrapy_df = df[['사업장명', "도로명전체주소"]]

In [95]:
scrapy_df['도로명전체주소'] = scrapy_df['도로명전체주소'].str.split(',', n=1).str[0]
scrapy_df

,사업장명,도로명전체주소
0,탑동에이치지,제주특별자치도 제주시 무근성안길 16
1,에릭스에스프레소,제주특별자치도 제주시 구좌읍 구좌로 77
2,일품순두부한림점,제주특별자치도 제주시 한림읍 한림상로 237
3,김복남맥주제주아라점,제주특별자치도 제주시 중앙로 581
4,봉플라봉뱅,제주특별자치도 제주시 문송1길 6-1
...,...,...
20651,못(MOAT),제주특별자치도 서귀포시 남원읍 태위로510번길 47-1
20652,제주하이루탕후루,제주특별자치도 서귀포시 중정로73번길 20
20653,연돈,제주특별자치도 서귀포시 색달로 10
20654,가시림 카페,제주특별자치도 서귀포시 표선면 녹산로5번길 171


In [96]:
scrapy_df1 = scrapy_df['사업장명'] +' '+ scrapy_df['도로명전체주소']
scrapy_df1


0                   탑동에이치지 제주특별자치도 제주시 무근성안길 16
1               에릭스에스프레소 제주특별자치도 제주시 구좌읍 구좌로 77
2             일품순두부한림점 제주특별자치도 제주시 한림읍 한림상로 237
3                김복남맥주제주아라점 제주특별자치도 제주시 중앙로 581
4                    봉플라봉뱅 제주특별자치도 제주시 문송1길 6-1
                          ...                  
20651    못(MOAT) 제주특별자치도 서귀포시 남원읍 태위로510번길 47-1
20652          제주하이루탕후루 제주특별자치도 서귀포시 중정로73번길 20
20653                    연돈 제주특별자치도 서귀포시 색달로 10
20654        가시림 카페 제주특별자치도 서귀포시 표선면 녹산로5번길 171
20655     올드타운 제주특별자치도 서귀포시 표선면 성읍정의현로22번길 18-8
Length: 20656, dtype: object

In [97]:
scrapy_df1
scrapy_df2

0            제주 탑동에이치지
1          제주 에릭스에스프레소
2          제주 일품순두부한림점
3        제주 김복남맥주제주아라점
4             제주 봉플라봉뱅
             ...      
20651       제주 못(MOAT)
20652      제주 제주하이루탕후루
20653            제주 연돈
20654        제주 가시림 카페
20655          제주 올드타운
Name: 사업장명, Length: 20656, dtype: object

In [98]:
# testData
scrapy_df1[17005:17006]

scrapy_df['사업장명'][17005:17006]



17005    제주와함께라면
Name: 사업장명, dtype: object

In [99]:
# 크롬 드라이버 실행
# chrome_options = webdriver.ChromeOptions()
# driver = webdriver.Chrome(options=chrome_options)
driver = webdriver.Chrome()
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [100]:
url = f"https://map.naver.com/v5/search/"
    # url = f"https://map.naver.com/v5/search/걸작떡볶이제주노형점 제주특별자치도 제주시 노형9길 7"
    # url = f"https://map.naver.com/v5/search/신촌도시락 제주 제주시 조천읍 신북로 88"
    # url = f"https://map.naver.com/v5/search/우본가해장국 제주특별자치도 제주시 건주로 72"
    
driver.get(url)

In [101]:
# for i in range(17001, 17011):
#     url = f"https://map.naver.com/v5/search/{scrapy_df['사업장명'][i]} {scrapy_df['도로명전체주소'][i]}"
# # driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[1]/div/div[4]/a/span[2]/svg').send_keys(Keys.ENTER) # 텍스트 전체 볼 수 있게 클릭

In [103]:
scrapy_df['사업장명'][70:80]

70      오토김밥제주점
71    젠틀래빗제주연동점
72         딜레탕트
73      제주울어멍김밥
74         체게바라
75          미리로
76      36.5도여름
77           더더
78         안녕전복
79       한앤솜하우스
Name: 사업장명, dtype: object

# 리뷰추출 extract_review

In [ ]:
                            
# '''
# listsCount = len(lists)의 갯수
#     --------2, 3 개일 때 ---------
#     별점 블로그 방문자 모두 O listsCount == 3
#     별점 X , 블로그 방문자 O listsCount == 2
    
#     --------1 개일 때 ---------
#     별점 X 블로그 X, 방문자O listsCount == 1
# ========================================    
# 크롤링해올 review 종류 : 

# 구버전
#     listsCount == 1 일 때
#         1. 사진 O  더보기 O
        
#         2. 사진 O  더보기 X : 6 2 3    3
#         //*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div/ul/li[4]/div/div[3]/a/span
#         3. 사진 X  더보기 O : 6 2 3    3 / 6 2 3 1    2
#         //*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div/ul/li[3]/div/div[3]/a/span
#         //*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li[1]/div/div[2]/a/span
#             더보기
#             //*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div/ul/li[3]/div/div[3]/a/span[2]
#             //*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li[1]/div/div[2]/a/span[2]
#         4. 사진 X  더보기 X : 6 2 3   2
#         //*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div/ul/li[6]/div/div[2]/a/span
        

#     listCount == 2,3일 때
#         1. 사진 O  더보기 O : 6 3 3 1   3
#         //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[30]/div/div[3]/a/span
#             더보기 : 6 3 3 1   3
#             //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[29]/div/div[3]/a

#         2. 사진 O  더보기 X : 6 3 3 1   3
#         //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[27]/div/div[3]/a/span

#         3. 사진 X  더보기 O : 6 3 3 1   2
#         //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[31]/div/div[2]/a/span
#             더보기 : 6 3 3 1  2
#             //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[31]/div/div[2]/a

#         4. 사진 X  더보기 X : 6 3 3 1   2
#         //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[26]/div/div[2]/a/span

# 신버전
#     listsCount == 1 일 때(옥성식당)
#         1. 사진 O  더보기 O : 6 3 3   4
#         //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[1]/div/div[4]/a/span
#             더보기 : 6 3 3   4
#             //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[4]/div/div[4]/a

#         2. 사진 O  더보기 X : 6 2 3 1  4  / 6 3 3   4
#         //*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li[1]/div/div[4]/a/span
#         //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[2]/div/div[4]/a/span

#         3. 사진 X  더보기 O : 6 2 3 1  3
#         //*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li[3]/div/div[3]/a/span
#         //*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li[4]/div/div[3]/a/span
#             더보기
#             //*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li[3]/div/div[3]/a
#             //*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li[4]/div/div[3]/a
        
#         4. 사진 X  더보기 X : 6 2 3    3
#         //*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div/ul/li[1]/div/div[3]/a/span

#     listCount == 2,3일 때 (제주와 함께라면)
#         1. 사진 O  더보기 O : 6 3 3 1   4
#             더보기 : 6 3 3 1   4
#             //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[2]/div/div[4]/a
#             //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[9]/div/div[4]/a
#         //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[2]/div/div[4]/a/span
#         //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[4]/div/div[4]/a/span
#         //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[5]/div/div[4]/a/span
#         //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[10]/div/div[4]/a/span
        
#         2. 사진 O  더보기 X : 6 3 3 1   4
#         //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[1]/div/div[4]/a/span
        
#         3. 사진 X  더보기 O 
#             더보기 : 6 3 3 1    3
#             //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[11]/div/div[3]/a
#         //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[11]/div/div[3]/a/span
        
#         4. 사진 X  더보기 X : 6 3 3 1   3
#         //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[6]/div/div[3]/a/span

# 사진이 있는 리뷰의 댓글과 사진이 없는 댓글의 경우 내용 자체가 다름. 


# 10개 넘어서 더보기 위해 '더보기'클릭
# listscount == 3
# //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a

# listcount == 2
# //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a

# //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a


# '''
# def extract_review(listsCount: int):
#     time.sleep(random.uniform(2,2.5))
#     rev = []
#     # 미리 '더보기' 버튼을 두번 눌러 range(1,31)로 늘려놓는다.
#     if listsCount == 1:
#             driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a').send_keys(Keys.ENTER)
#             time.sleep(0.1)
#             driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a').send_keys(Keys.ENTER)
#             time.sleep(0.1)
#     else:
#         driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a').send_keys(Keys.ENTER)
#         time.sleep(0.1)
#         driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a').send_keys(Keys.ENTER)
#         time.sleep(0.1)

        
#     # 처음부터 끝까지 리뷰 추출
#     for i in range(1,31):
#         # listsCount == 1     
#         if listsCount == 1:
#             try: 
#                 # 텍스트 전체 볼 수 있게 클릭 구버전
#                 driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div/div[3]/a').send_keys(Keys.ENTER)
#                 time.sleep(0.1)
#                 # 리뷰 text
#                 comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div/div[3]/a').text
#                 rev.append(comment)
#         # 사진X

# 초기에는 크롤링 할 데이터를 전부 XPATH 로 진행하였으나 Xpath에는 예외가 너무 많음

# HTML, class로 크롤링 하기로 변경

In [72]:
scrapy_df1[17005:17006]

17005    제주와함께라면 제주특별자치도 제주시 애월읍 애월해안로 274
dtype: object

In [135]:
# 연습용 ( 테스트용 )
t = random.randrange(1,2)
# 대기를 위한 WebDriverWait 객체 생성
wait = WebDriverWait(driver, t)

for index, scrapyList in enumerate(scrapy_df1[17005:17106], start=17005):
    url1 = f"https://map.naver.com/v5/search/{scrapy_df1[index]}"
    url2 = f"https://map.naver.com/v5/search/{scrapy_df2[index]}"
    # url = f"https://map.naver.com/v5/search/걸작떡볶이제주노형점 제주특별자치도 제주시 노형9길 7"
    # url = f"https://map.naver.com/v5/search/신촌도시락 제주 제주시 조천읍 신북로 88"
    # url = f"https://map.naver.com/v5/search/우본가해장국 제주특별자치도 제주시 건주로 72"
    
    driver.get(url1)
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    driver.switch_to.frame(driver.find_element(By.XPATH, '//*[@id="entryIframe"]')) # iframe 이동
    # 첫 번째 요소가 존재하는지 확인 후 가져오기
    first_elements = soup.select('span.PXMot a')
    if first_elements:
        first_text = first_elements[0].text.strip()
        print(first_text)
    else:
        print("첫 번째 리뷰가 존재하지 않습니다.")


    # if reviewSelector:
    #     for span in reviewSelector:
    #         print(span.text)
    # else:
    #     print("리뷰를 찾을 수 없습니다.")
    titleSelector = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="_title"]/div/span[1]')))
    # starsSelector = soup.find('span', class_='PXMot LXIwF')
    
    before_title = titleSelector.text
    print(before_title)

    # starsSelector = soup.select('.PXMot.LXIwF')
    # starsSelector = driver.find_element(By.XPATH, '')
    
    try:
        starsSelector = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'PXMot.LXIwF')))
        stars_ = starsSelector.text
        print({stars_})
    except:
        print("별점없음")
            # 방문자리뷰, 블로그리뷰
        try:
            reviewSelector = soup.select('PXMot') 
            print(f"{reviewSelector}찾음")
            # if reviewSelector:
            #     if len(reviewSelector) == 3:
            #         welcomeSelector = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(2) > a')
            #         blogSelector = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(3) > a')
            #         welcomeReviewCount_ = welcomeSelector.text.strip()
            #         print(f"방문자리뷰 : {welcomeReviewCount_}")
            #         blogReviewCount_ = blogSelector.text.strip()
            #         print(f"블로그리뷰 : {blogReviewCount_}")
            #         time.sleep(1)
            #     elif len(reviewSelector) == 2:
            #         welcomeSelector = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(1) > a')
            #         blogSelctor = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(2) > a')
            #         welcomeReviewCount_ = welcomeSelector.text.strip()
            #         print(f"방문자리뷰 : {welcomeReviewCount_}")
            #         blogReviewCount_ = blogSelctor.text.strip()
            #         print(f"블로그리뷰 : {blogReviewCount_}")
            #     else:
            #         welcomeSelector = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span > a')
            #         welcomeReviewCount_ = welcomeSelector.text.strip()
            #         print(f"방문자리뷰 : {welcomeReviewCount_}")
            #         blogReviewCount_ = "X"
            #         print(f"블로그리뷰 : {blogReviewCount_}")
            # else:
            #     print("방문자리뷰, 블로그리뷰 못찾음")
        except Exception as e:
            print("리뷰 파싱 에러")
        
        # 방문자리뷰 클릭 => 최신순 클릭 => 더보기 클릭 => 더보기 클릭
        time.sleep(3)


첫 번째 리뷰가 존재하지 않습니다.
제주와함께라면
{'별점\n4.45'}


KeyboardInterrupt: 

In [124]:
scrapy_df1[17005:17006]

# ===== 크롤링 설정 =====
idx = 0 # 크롤링 총 갯수
failed_title_idx = 0 # title 크롤링 실패시 idx 1 증가
failed_stars_idx = 0 # stars 크롤링 실패시 idx 1 증가
failed_Review_idx = 0

header_count = 0 # header파트에 (별점, 방문자리뷰, 블로그리뷰) == 0 or 1 or 2 or 3 

t = random.randrange(1,2)
# 대기를 위한 WebDriverWait 객체 생성
wait = WebDriverWait(driver, t)


# 크롤링한 데이터 담길곳
scrapy_header_data = pd.DataFrame(columns=['Title', 'titleDetail', 'stars', 'welcomReviewCount','blogReviewCount'])

# 크롤링중 에러발생 데이터 담길 곳. 
'''
1. notSearche : 데이터를 찾지 못하였을 때
2. Title : 제목관련 데이터 찾지 못했을 때
3. stars : 별점 관련 데이터 없을 때
4. ReviewCount : 리뷰데이터 없을 때
'''
scrapy_error_data = pd.DataFrame(columns=['notSearch', 'Title', 'stars', 'ReviewCount'])

'''
크롤링 기준: dfList1 검색 
if dfList1 can not find iframe :
    use dfList2
'''

# ===== 크롤링 해올 데이터 담길 배열 ======
title = [] #이름 담길 배열 (div class = 'Fc1rA) // EX: 마야블루
titleDetail = [] #이름 옆에 분류 담길 배열 (div class = 'DJJvD) // EX: 패션잡화
stars = [] #별점 담길 배열 (별점은 있을 수도, 없을 수도 있음. 예외처리필요)
welcomeReviewCount = [] #방문자 리뷰 (PXMot)
blogReviewCount = [] #블로그 리뷰 (PXMot)

# ===== Error Log 담길 배열 =====
error_notSearch = [] # = 검색결과 없음
error_title = []
error_stars = []
error_ReviewCount = []

def add_error_data(error_type, index, title):
    if error_type == 'notSearch':
        error_notSearch.append({"error_index": index, 'error_name': title})
    elif error_type == 'title':
        error_title.append({"error_index": index, "error_title": title})
    elif error_type == 'stars':
        error_stars.append({"error_index": index, "error_title": title})
    elif error_type == 'ReviewCount':
        error_ReviewCount.append({"error_index": index, "error_title": title})
        
# 방문자 리뷰 클릭 후 그 이후 과정 (최신순 버튼 클릭 => 더보기 버튼 두번 클릭 )
def button_click_header_next():
    # 최신순 버튼 클릭
    late_button = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/div[2]/div[1]/span[2]/a')))
    late_button.click()
    time.sleep(0.1)

    # 더보기 버튼 두번 클릭 (더보기 버튼이있다면)
    try:
        more_button = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a')))
        if more_button:
            more_button.click()
            time.sleep(0.1)
            more_button.click()
            time.sleep(0.1)
    except Exception as e:
        pass

# 방문자 리뷰 클릭 버튼
def button_click_header(header_count : int):
        if header_count == 1:
            # 방문자 리뷰 버튼 클릭
            come_review_button = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span/a')))
            come_review_button.click()  # 클릭 이벤트 발생
        elif header_count == 2:
            # 방문자 리뷰 버튼 클릭
            come_review_button = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]/a')))
            come_review_button.click()  # 클릭 이벤트 발생
        elif header_count == 3:
            come_review_button = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[2]/a')))
            come_review_button.click()  # 클릭 이벤트 발생
        else:
            print("버튼클릭 못함.")
            
            
        time.sleep(1)




for index, scrapyList in enumerate(scrapy_df1[17005:17066], start=17005):
    url1 = f"https://map.naver.com/v5/search/{scrapy_df1[index]}"
    url2 = f"https://map.naver.com/v5/search/{scrapy_df2[index]}"
    # url = f"https://map.naver.com/v5/search/걸작떡볶이제주노형점 제주특별자치도 제주시 노형9길 7"
    # url = f"https://map.naver.com/v5/search/신촌도시락 제주 제주시 조천읍 신북로 88"
    # url = f"https://map.naver.com/v5/search/우본가해장국 제주특별자치도 제주시 건주로 72"
    
    driver.get(url1)
    idx += 1
    print(f"====================번호{idx}==================")
    time.sleep(3)
    # ====================Fucntion=========================
    # 버튼클릭 함수(방문자 리뷰 => 최신순 => 더보기 => 더보기)
    
    '''
    try:
        url1으로 도전: 사업장명 + 주소
    except:
        url2로 도전: 제주 + 사업장명

    '''
    # url1으로 들어가 항목이 있는지 체크, 별점, 방문자리뷰, 블로그리뷰 갯수만 크롤링
    try:
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        driver.switch_to.frame(driver.find_element(By.XPATH, '//*[@id="entryIframe"]')) # iframe 이동

        # title, titleDetail
        try:
            # title, titleDetail
            titleSelector = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="_title"]/div/span[1]')))
            titleDetailSelector = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="_title"]/div/span[2]')))

            # title, titleDetail가져오기
            # title
            before_title = titleSelector.text
            print(f"title : {before_title}")
            title.append(before_title)
            # titleDetail   
            before_titleDetail = titleDetailSelector.text
            print(f"상세분류 : {before_titleDetail}")
            titleDetail.append(before_titleDetail)
            time.sleep(1)
        except Exception as e:
            add_error_data('title', index, scrapyList)
            failed_title_idx += 1
            print("===========title 크롤링 실패")

        # 별점
        try:
            starsSelector = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'PXMot.LXIwF')))
            
            stars_ = starsSelector.text
            # 텍스트에서 "별점" 제거하고 공백을 기준으로 분리하여 첫 번째 요소를 선택하여 숫자값만 저장
            stars_ = stars_.split()[1]
            stars.append(stars_)
            print(f"별점 : {stars_}")
                
        except Exception as e:
            add_error_data('stars', index, scrapyList)
            stars.append("X")
            print("별점 없음")
            failed_stars_idx += 1

        # 방문자리뷰, 블로그리뷰
        try:
            reviewSelector = soup.find_all('span', class_='PXMot')
            if reviewSelector:
                header_count = len(reviewSelector)
                if header_count == 3:
                    welcomeSelector = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(2) > a')
                    blogSelector = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(3) > a')
                    welcomeReviewCount_ = welcomeSelector.text.strip()
                    print(f"방문자리뷰 : {welcomeReviewCount_}")
                    blogReviewCount_ = blogSelector.text.strip()
                    print(f"블로그리뷰 : {blogReviewCount_}")
                    time.sleep(1)
                elif header_count  == 2:
                    welcomeSelector = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(1) > a')
                    blogSelctor = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(2) > a')
                    welcomeReviewCount_ = welcomeSelector.text.strip()
                    print(f"방문자리뷰 : {welcomeReviewCount_}")
                    blogReviewCount_ = blogSelctor.text.strip()
                    print(f"블로그리뷰 : {blogReviewCount_}")
                elif header_count == 1:
                    welcomeSelector = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span > a')
                    welcomeReviewCount_ = welcomeSelector.text.strip()
                    print(f"방문자리뷰 : {welcomeReviewCount_}")
                    blogReviewCount_ = "X"
                    print(f"블로그리뷰 : {blogReviewCount_}")
                welcomeReviewCount.append(welcomeReviewCount_)        
                blogReviewCount.append(blogReviewCount_)
                
                # 방문자리뷰 클릭 => 최신순 클릭 => 더보기 클릭 => 더보기 클릭
                button_click_header(header_count)
            else:
                print("방문자리뷰, 블로그리뷰 못찾음")
                welcomeReviewCount.append("None")
                blogReviewCount.append("None")
        except Exception as e:
            add_error_data('ReviewCount', index, scrapyList)
            print("리뷰 파싱 에러")
            failed_Review_idx += 1
        try:
            reviewSelector = soup.find('span', class_='PXMot')
            if reviewSelector:
                # reviewSelector가 None이 아니라면 처리
                welcomeSelector = reviewSelector.find('a', href="/restaurant/37840644/review/visitor")
                blogSelector = reviewSelector.find('a', href="/restaurant/37840644/review/blog")
                
                if welcomeSelector and blogSelector:
                    welcomeReviewCount_ = welcomeSelector.text.strip()
                    blogReviewCount_ = blogSelector.text.strip()
                    print(f"방문자리뷰 : {welcomeReviewCount_}")
                    print(f"블로그리뷰 : {blogReviewCount_}")
                elif welcomeSelector:
                    welcomeReviewCount_ = welcomeSelector.text.strip()
                    print(f"방문자리뷰 : {welcomeReviewCount_}")
                    blogReviewCount_ = "X"
                    print(f"블로그리뷰 : {blogReviewCount_}")
                else:
                    print("방문자리뷰, 블로그리뷰 못찾음")
                    welcomeReviewCount_ = "None"
                    blogReviewCount_ = "None"
                
                welcomeReviewCount.append(welcomeReviewCount_)        
                blogReviewCount.append(blogReviewCount_)
                
                # 방문자리뷰 클릭 => 최신순 클릭 => 더보기 클릭 => 더보기 클릭
                button_click_header(header_count)
            else:
                print("방문자리뷰, 블로그리뷰 못찾음")
                welcomeReviewCount.append("None")
                blogReviewCount.append("None")
        except Exception as e:
            add_error_data('ReviewCount', index, scrapyList)
            print("리뷰 파싱 에러")
            failed_Review_idx += 1

        
        
        

    # URL1 감지 불가 URL2로 시도.
    except NoSuchElementException:
        print(f"url2로 검색 시도")
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        # driver.switch_to.frame(driver.find_element(By.XPATH, '//*[@id="searchIframe"]'))
        driver.get(url2)
        time.sleep(2)
        try:
            driver.switch_to.frame(driver.find_element(By.XPATH, '//*[@id="entryIframe"]')) # iframe 이동
            print(f'{index}업체 있음.')
            time.sleep(2)
            
        except NoSuchElementException:
            print(f"{index} - 2 조건에 맞는 업체가 없습니다.")
            continue


        
        

    

====================번호1==================
title : 제주와함께라면
상세분류 : 라면
별점 : 4.45
방문자리뷰, 블로그리뷰 못찾음
====================번호2==================
title : 만사옥게
상세분류 : 농수산물
별점 없음
방문자리뷰, 블로그리뷰 못찾음
====================번호3==================


KeyboardInterrupt: 

In [107]:
count = 0 # 성공한 크롤링 카운트
failedCount = 0 # 실패한 크롤링 카운트
# scrapy_df1 : '사업장명' + '주소'
# scrapy_df2 : '제주' + '사업장명'

crawlingReviewData = [] # 식당 리뷰를 저장할 리스트
crawlingImageData = [] # 식당 이미지를 저장할 리스트
crawlingCategoryData = [] # 식당 업종분류를 저장할 리스트
crawlingVisitCountData = [] # 식당 방문자리뷰 수를 저장할 리스트
crawlingBlogCountData = [] # 식당 블로그리뷰 수를 저장할 리스트
crawlingRatingData = [] # 식당 평점을 저장할 리스트
errorResult = ""

result_df = pd.DataFrame()# 결과저장
# 테스트 데이터1 :(별점O, 방문자O, 블로그O) 제주와 함께라면 : listcount ==  3
# 테스트 데이터2 :(별점X, 방문자O, 블로그O) 만사옥게 : listcount == 2
# 테스트 데이터3 :(별점X, 방문자O, 블로그X) 옥성식당 : listcount == 1
for index, scrapyList in enumerate(scrapy_df1[17005:17006], start = 17005):
    url1 = f"https://map.naver.com/v5/search/{scrapy_df1[index]}"
    url2 = f"https://map.naver.com/v5/search/{scrapy_df2[index]}"
    # url = f"https://map.naver.com/v5/search/걸작떡볶이제주노형점 제주특별자치도 제주시 노형9길 7"
    # url = f"https://map.naver.com/v5/search/신촌도시락 제주 제주시 조천읍 신북로 88"
    # url = f"https://map.naver.com/v5/search/우본가해장국 제주특별자치도 제주시 건주로 72"
    
    driver.get(url1)
    print(f"=================url1 {index}시도=================")
    time.sleep(random.uniform(2.0, 2.5))
    try: 
        driver.switch_to.frame(driver.find_element(By.XPATH, '//*[@id="entryIframe"]')) # iframe 이동
        time.sleep(random.uniform(1.5, 2.0))
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        print(f'"=================url1 {index}찾음=================')
    except NoSuchElementException as e:
        driver.get(url2)
        print(f"=================url2 {index}시도=================")
        time.sleep(random.uniform(2.0, 2.5))
        try:
            driver.switch_to.frame(driver.find_element(By.XPATH, '//*[@id="entryIframe"]')) # iframe 이동
            time.sleep(random.uniform(1.5, 2.0))
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            print("=================url2 {index}찾음=================")
        except NoSuchElementException as e:
            print(f"{index} 검색결과를 찾을 수 없음.")
            continue

        # 이미지 3장 뽑아서 저장하기
        images = []  # 이미지 링크를 저장할 리스트
        for i in range(1, 4):
            image_element = soup.select_one(f"#ibu_{i}")  # ibu_1 ~ 3 이런식으로 이미지의 id가 갯수별로 형성됨
            if not image_element:
                continue  # 이미지 요소가 없으면 for 루프 종료
            style = image_element["style"]
            background_image = style.split("url(")[1].split(")")[0]
            images.append(background_image)  # 이미지 링크를 리스트에 추가

        # 음식 카테고리 뽑기 (버튼이라서 CSS_SELECTOR를 써야함)
        foodCategory = driver.find_element(By.CSS_SELECTOR, '#_title > div > span.DJJvD').text


        # 리뷰 탭을 선택하기 전에 버튼 갯수 파악하기 (별점 4.7, 방문자리뷰 100, 블로그리뷰 120) 이런식으로 된곳
        lists = soup.select('.PXMot')  # 버튼들의 class = PXMot
        errorResult = "검색 결과 없음"
        '''
        ratingText = 별점
        visitText = 방문자리뷰 갯수
        blogText = 블로그리뷰 갯수
        '''
        if len(lists) == 3: # (별점,블로그,방문자 있는 경우) 별점/방문자리뷰/블로그리뷰 순일때 방문자리뷰는 두번째에 위치=span[2]
            # 별점 , 방문자리뷰 갯수, 블로그리뷰 갯수 긁어오기
            ratingText = soup.select_one('.PXMot.LXIwF').text # 별점의 class = .PXMot.LXIwF
            visitText = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(2) > a').text
            blogText = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(3) > a').text
            

            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[2]/a').send_keys(Keys.ENTER) # 방문자 리뷰 클릭
            time.sleep(1)
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/div[2]/div[1]/span[2]/a').send_keys(Keys.ENTER) # 최신순으로 클릭
            time.sleep(1)
        elif len(lists) == 2: # (별점이 없는 경우) 방문자리뷰/블로그리뷰 순일때 방문자리뷰는 첫번째에 위치=span[1]
            # 리뷰 수 긁어오기                                 
            visitText = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(1) > a').text
            blogText = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(2) > a').text
            ratingText = "x"

            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]/a').send_keys(Keys.ENTER) # 방문자 리뷰 클릭
            time.sleep(1)
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/div[2]/div[1]/span[2]/a').send_keys(Keys.ENTER) # 최신순으로 클릭
            time.sleep(1)
        elif len(lists) == 1: # 블로그 리뷰가 없는 경우 
            visitText = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span > a').text
            blogText = "블로그리뷰 0"
            ratingText = "x"

            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span/a').send_keys(Keys.ENTER) # 방문자 리뷰 클릭
            time.sleep(1)
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/div[2]/div[1]/span[2]/a').send_keys(Keys.ENTER) # 최신순으로 클릭
            time.sleep(1) 
        else:
            errorResult = "크롤링 불가"
# 여기까지가 이제 방문자리뷰 들어가서 최신순 클릭한 상태.

# 리뷰 추출 함수 소환
        review = extract_review(len(lists))
########
        crawlingReviewData.append(review)
        crawlingImageData.append(images)
        crawlingCategoryData.append(foodCategory)
        crawlingRatingData.append(ratingText.replace("별점", ""))
        crawlingVisitCountData.append(int(re.sub(r'\D', '', visitText)))  # 정규표현식을 사용하여 숫자만 추출
        crawlingBlogCountData.append(int(re.sub(r'\D', '', blogText)))
        count += 1
        print("----------------------------------")
        print(f'크롤링 성공! 성공: {count}, 실패: {failedCount}')
        ## 별점이 있고 없고에 따라 공백 데이터 추가해주는거 필요
    except:
        crawlingReviewData.append('x')
        crawlingImageData.append('x')
        crawlingCategoryData.append('x')
        crawlingRatingData.append('x')
        crawlingVisitCountData.append("x")
        crawlingBlogCountData.append("x")
        failedCount += 1
        print("----------------------------------")
        print(f'크롤링 실패! 성공: {count}, 실패: {failedCount}, 이유: {errorResult}')

    result_df = pd.DataFrame({'foodCategory': crawlingCategoryData, 'images': crawlingImageData, 'rating': crawlingRatingData, 'visitReviewCount': crawlingVisitCountData, 'blogReviewCount': crawlingBlogCountData,'review': crawlingReviewData})


1~10 try (else) 사진ㅇreview1
1~10 try (else) 사진x review1
1~10 try (else) 사진ㅇreview2
1~10 try (else) 사진x review2
1~10 try (else) 사진ㅇreview3
1~10 try (else) 사진x review3
1~10 try (else) 사진ㅇreview4
1~10 try (else) 사진x review4
1~10 try (else) 사진ㅇreview5
1~10 try (else) 사진x review5
1~10 try (else) 사진ㅇreview6
1~10 try (else) 사진x review6
1~10 try (else) 사진ㅇreview7
1~10 try (else) 사진x review7
1~10 try (else) 사진ㅇreview8
1~10 try (else) 사진x review8
1~10 try (else) 사진ㅇreview9
1~10 try (else) 사진x review9
1~10 try (else) 사진ㅇreview10
1~10 try (else) 사진x review10
더보기 (11~20)
더보기 클릭!else
11~20 try (else) 더보기 사진ㅇreview 11
11~20 try (else) 더보기 사진x review 11
11~20 try (else) 더보기 사진ㅇreview 12
11~20 try (else) 더보기 사진x review 12
11~20 try (else) 더보기 사진ㅇreview 13
11~20 try (else) 더보기 사진x review 13
11~20 try (else) 더보기 사진ㅇreview 14
11~20 try (else) 더보기 사진x review 14
11~20 try (else) 더보기 사진ㅇreview 15
11~20 try (else) 더보기 사진x review 15
11~20 try (else) 더보기 사진ㅇreview 16
11~20 try (else) 더보기 사진x review 16
11~20 try 

In [108]:
result_df

['"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20220725_27%2F16587221512078HtLn_JPEG%2FScreenshot_20220725-130130_Gallery.jpg"',
 '"https://search.pstatic.net/common/?autoRotate=true&type=w278_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20220725_46%2F1658722228586sIzpT_JPEG%2FScreenshot_20220302-174825_NAVER.jpg"',
 '"https://search.pstatic.net/common/?autoRotate=true&type=w278_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20220725_139%2F16587221704175fwWb_JPEG%2FScreenshot_20220725-130639_NAVER.jpg"']

In [109]:
foodCategory

'라면'

In [ ]:
# 최신순 버튼 클릭 못하네
driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/div[2]/div[1]/span[2]/a').send_keys(Keys.ENTER) # 최신순으로 클릭
                          #    //*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/div[2]/div[1]/span[2]/a 
                                                                    #  ㄴ기존 코드랑 달라진 부분

In [ ]:
###  본인작성 리뷰만 가져오자. 자동작성 리뷰는 긍정뿐이 없음.
### 사진 x
# 2번리뷰 (사진 X, 본인작성 리뷰, 상세한 리뷰) - 상세한 리뷰 ( 이용방법, 대기시간, 목적, 동행 등을 입력한 리뷰 )
driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[2]/div/div[3]/a/span').text 
# 6번리뷰 (사진 X, 본인작성 리뷰, 기존 리뷰 ) - 기존리뷰
driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[6]/div/div[2]/a/span').text
# 8번리뷰 (사진 X, 본인작성 리뷰, 기존 리뷰 )
driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[8]/div/div[2]/a/span').text
# 9번리뷰 (사진 X, 본인작성 리뷰, 기존 리뷰 )
driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[9]/div/div[2]/a/span').text
# 10번리뷰 (사진 X, 본인작성 리뷰, 기존 리뷰 )
driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[10]/div/div[2]/a/span').text

### 사진 o
# 3번 리뷰 (사진 O, 본인작성 리뷰)
driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[3]/div/div[4]/a/span').text
# 4번 리뷰 (사진 O, 본인작성 리뷰)
driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[4]/div/div[4]/a/span').text


# driver.find_element(By.XPATH, '').text
# driver.find_element(By.XPATH, '').text
# driver.find_element(By.XPATH, '').text
# driver.find_element(By.XPATH, '').text
# driver.find_element(By.XPATH, '').text
# driver.find_element(By.XPATH, '').text
# driver.find_element(By.XPATH, '').text



NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[2]/div/div[3]/a/span"}
  (Session info: chrome=122.0.6261.112); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000100d30508 chromedriver + 3966216
1   chromedriver                        0x0000000100d28a94 chromedriver + 3934868
2   chromedriver                        0x00000001009abda0 chromedriver + 277920
3   chromedriver                        0x00000001009ee394 chromedriver + 549780
4   chromedriver                        0x0000000100a26bf0 chromedriver + 781296
5   chromedriver                        0x00000001009e2fb0 chromedriver + 503728
6   chromedriver                        0x00000001009e3a28 chromedriver + 506408
7   chromedriver                      

In [ ]:
# 10번리뷰 (사진 X, 본인작성 리뷰, 기존 리뷰 )
driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[10]/div/div[2]/a').text

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[10]/div/div[2]/a"}
  (Session info: chrome=122.0.6261.112); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000100d30508 chromedriver + 3966216
1   chromedriver                        0x0000000100d28a94 chromedriver + 3934868
2   chromedriver                        0x00000001009abda0 chromedriver + 277920
3   chromedriver                        0x00000001009ee394 chromedriver + 549780
4   chromedriver                        0x0000000100a26bf0 chromedriver + 781296
5   chromedriver                        0x00000001009e2fb0 chromedriver + 503728
6   chromedriver                        0x00000001009e3a28 chromedriver + 506408
7   chromedriver                        0x

In [ ]:

# driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a').send_keys(Keys.ENTER)
# //*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a/span

driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a/span').send_keys(Keys.ENTER)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a/span"}
  (Session info: chrome=122.0.6261.112); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000100d30508 chromedriver + 3966216
1   chromedriver                        0x0000000100d28a94 chromedriver + 3934868
2   chromedriver                        0x00000001009abda0 chromedriver + 277920
3   chromedriver                        0x00000001009ee394 chromedriver + 549780
4   chromedriver                        0x0000000100a26bf0 chromedriver + 781296
5   chromedriver                        0x00000001009e2fb0 chromedriver + 503728
6   chromedriver                        0x00000001009e3a28 chromedriver + 506408
7   chromedriver                        0x0000000100cf

In [13]:
lists = soup.select('.PXMot')  # 버튼들의 class = PXMot
errorResult = "검색 결과 없음"
if len(lists) == 3: # (별점이 있는 경우) 별점/방문자리뷰/블로그리뷰 순일때 방문자리뷰는 두번째에 위치=span[2]
    # 별점 수, 리뷰 수 긁어오기
    ratingText = soup.select_one('.PXMot.LXIwF').text # 별점의 class = .PXMot.LXIwF
    visitText = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(2) > a').text
    blogText = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(3) > a').text

    driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[2]/a').send_keys(Keys.ENTER) # 방문자 리뷰
    time.sleep(1)
    driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/div[2]/div[1]/span[2]/a').send_keys(Keys.ENTER) # 최신순으로 클릭
    time.sleep(1)
elif len(lists) == 2: # (별점이 없는 경우) 방문자리뷰/블로그리뷰 순일때 방문자리뷰는 첫번째에 위치=span[1]
    # 리뷰 수 긁어오기                                 
    visitText = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(1) > a').text
    blogText = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(2) > a').text
    ratingText = "x"

    driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]/a').send_keys(Keys.ENTER) # 방문자 리뷰 클릭
    time.sleep(1)
    driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/div[2]/div[1]/span[2]/a').send_keys(Keys.ENTER) # 최신순으로 클릭
    time.sleep(1)
elif len(lists) == 1: # 블로그 리뷰가 없는 경우 
    visitText = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span > a').text
    blogText = "블로그리뷰 0"
    ratingText = "x"

    driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span/a').send_keys(Keys.ENTER) # 방문자 리뷰 클릭
    time.sleep(1)
    driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/div[2]/div[1]/span[2]/a').send_keys(Keys.ENTER) # 최신순으로 클릭
    time.sleep(1) 
else:
    errorResult = "크롤링 불가"
###########
# 위 내용이 최신순까지 클릭한 상태.
# 아래 내용은 extract_review를 정의하고 review만 가져오는 함수.
###########    
'''
크롤링해올 review 종류 : 
구버전
    1. 사진 O 내용 O 더보기 O
    2. 사진 O 내용 O 더보기 X
    3. 사진 X 내용 O 더보기 O 
    4. 사진 X 내용 O 더보기 X
신버전
    1. 사진 O 내용 O 더보기 O
    2. 사진 O 내용 O 더보기 X
    3. 사진 X 내용 O 더보기 O
    4. 사진 X 내용 O 더보기 X

사진이 있는 리뷰의 댓글과 사진이 없는 댓글의 경우 내용 자체가 다름. 
더보기버튼은 if문으로 뚫자.
for

'''

def extract_review(listsCount: int):
            time.sleep(random.uniform(2,2.5))
            rev = []
            for i in range(1,11):
                try:
                    # 사진 O 구버전
                    if listsCount == 1:
                        # 텍스트 전체 볼 수 있게 클릭
                        driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div/div[3]/a').send_keys(Keys.ENTER)
                        time.sleep(0.1)
                        # 리뷰 text
                        comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div/div[3]/a').text
                        rev.append(comment)
                except:
                     
                    # 사진X


review = extract_review(len(lists))